**Employee Importance**

You are given a data structure of employee information, which includes the employee's unique id, his importance value and his direct subordinates' id.

For example, employee 1 is the leader of employee 2, and employee 2 is the leader of employee 3. They have importance value 15, 10 and 5, respectively. Then employee 1 has a data structure like [1, 15, [2]], and employee 2 has [2, 10, [3]], and employee 3 has [3, 5, []]. Note that although employee 3 is also a subordinate of employee 1, the relationship is not direct.

Now given the employee information of a company, and an employee id, you need to return the total importance value of this employee and all his subordinates.

Example 1:
```
Input: [[1, 5, [2, 3]], [2, 3, []], [3, 3, []]], 1
Output: 11
Explanation:
Employee 1 has importance value 5, and he has two direct subordinates: employee 2 and employee 3. They both have importance value 3. So the total importance value of employee 1 is 5 + 3 + 3 = 11.
```
Ok so first thing's first, we need a better way to access each employee than looping through and looking for the right id each time. so we should set up a dictionary of employee.id : employee for each employee in employees. This set up will take O(n) but enable O(1) look up afterward. 

Then we can do a simple search, either bfs or dfs, while keeping track of the total importance of each employee that ends up enqueued/stacked. 

The runtime will be the number of subordinates of the target employee. Because there's no way to tell how many that will be, the general runtime will be O(n) where n = number of total employees. Same goes for the space complexity of the queue. 

In [ ]:
"""
# Employee info
class Employee:
    def __init__(self, id: int, importance: int, subordinates: List[int]):
        # It's the unique id of each node.
        # unique id of this employee
        self.id = id
        # the importance value of this employee
        self.importance = importance
        # the id of direct subordinates
        self.subordinates = subordinates
"""
class Solution:
    def getImportance(self, employees: List['Employee'], id: int) -> int:
        empMap = {emp.id: emp for emp in employees}
        if id not in empMap: return
        target = empMap[id]
        total = 0
        q = [target]
        while q:
            curr = q.pop(0)
            total += curr.importance
            for sub in curr.subordinates:
                q.append(empMap[sub])
        return total

**Oranges Rotting**

In a given grid, each cell can have one of three values:

- the value 0 representing an empty cell;
- the value 1 representing a fresh orange;
- the value 2 representing a rotten orange.
- Every minute, any fresh orange that is adjacent (4-directionally) to a rotten orange becomes rotten.

Return the minimum number of minutes that must elapse until no cell has a fresh orange.  If this is impossible, return -1 instead.

Hmmm. This seems like almost an inverse bfs, because we want to track all the fresh oranges rather than the rotten ones, but the rotten ones are behaving like a bfs. Every minute, we need to go through all of the fresh oranges and check if they've been rotted. But we can't immediately update them because that will affect the other oranges too soon. so at each minute, we need to loop through all the fresh oranges, track which oranges to rot, then rot them. Once they've rotted, we can remove them from the list of fresh oranges to check. Once there are no more oranges to check, we can return the number of minutes it's been. If at any point while we're looping, no new oranges have been added to rot but there are more to check, we know that we can't rot all the oranges and can return -1. So we just need to figure out how to loop through the grid with guards against exceeding the length of the lists. 

The runtime on this is long. Suppose n is the number of squares in the grid. first we have a O(n) to populate `toCheck`. but it's an addition to the rest, not a multiplication. Then at each 'minute', we loop through all the squares with 1s in them. We check the 4 squares directly around them, but that's a constant. This process gets shorter each time through but not in a consistent way, so this is also O(n). Then we loop through all the ones that have rotted, but that amount is affected directly by the size of `toCheck`, so this basically completes the O(n) of looping through `toCheck`. It's weird, we're checking more oranges than we have to each time because we should only be checking the ones touching rotting oranges, but leetcode says this answer is faster than 93% of the other submissions. But say the grid is one row with one rotten orange at the start and 1000 fresh ones. we'll have to go through all the fresh ones 1000 times, even as the inner loop gets smaller by one each time. Pretty sure that's O(n^2) worst case...

In [ ]:
class Solution:
    def orangesRotting(self, grid: List[List[int]]) -> int:
        mins = 0
        n = len(grid)
        m = len(grid[0])
        toCheck = set()
        for x in range(n):
            for y in range(m):
                if grid[x][y] == 1:
                    toCheck.add((x,y))
        dirs = [(1,0), (-1,0), (0,1), (0,-1)]
        while len(toCheck) > 0:
            toChange = set()
            for x, y in toCheck:
                for xD, yD in dirs:
                    if 0 <= x + xD < n and 0 <= y + yD < m and grid[x+xD][y+yD] == 2:
                        toChange.add((x,y))
                        break
            if len(toChange) == 0: return -1
            for x, y in toChange:
                grid[x][y] = 2
            toCheck = toCheck - toChange
            mins += 1
        return mins

**Course Schedule**

There are a total of n courses you have to take, labeled from 0 to n-1.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: `[0,1]`

Given the total number of courses and a list of prerequisite pairs, is it possible for you to finish all courses?

Note:
The input prerequisites is a graph represented by a list of edges, not adjacency matrices. Read more about how a graph is represented.
You may assume that there are no duplicate edges in the input prerequisites.

Is a node still a node by any other data type? Ok so each pair in the prereqs is a directed edge from the second element to the first, because for example in `[0,1]` above, you cannot take class 0 until you've first visited class 1. This problem is asking for a topological sort that returns false if there are any cycles. Which requires a dfs search, and to do that we want to access all of a node's outedges at a single time (for the for loop in the dfs). We need to organize our list of edges into a structure that we can easily access this way. I'm thinking a dictionary where each node has a list of neighbors it points to. nodes without neighbors can be saved with empty lists in the dict for easier access. our original 'list of verticies' is literally just range(n). And other than that, it's a classic topological sort search for cycles, just switching the fact that the main function should return true if there _isn't_ a cycle, and false if there _is_. 

Runtime is same as a regular dfs plus an extra runthroughs of size V (# of vertices) and an extra runthrough of E (# of edges) to build the prereq dict, but in an additive way, not multiplicative, so it doesn't add to the overall big O. so that's O(V + E). Space complexity is O(V + E) for the prereq dict plus the recursion stack, but it comes down to the same thing as well.

In [ ]:
class Solution:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        self.status = {}
        self.VISITING = -1
        self.VISITED = 0
        self.preqs = {i: [] for i in range(numCourses)}
        for pair in prerequisites:
            self.preqs[pair[1]].append(pair[0])
        for i in range(numCourses):
            if self._hasCycle(i):
                return False
        return True
    
    def _hasCycle(self, i):
        if i in self.status:
            if self.status[i] == self.VISITING:
                return True
            return False
        self.status[i] = self.VISITING
        for nei in self.preqs[i]:
            if self._hasCycle(nei):
                return True
        self.status[i] = self.VISITED
        return False

**Find Eventual Safe States**

In a directed graph, we start at some node and every turn, walk along a directed edge of the graph.  If we reach a node that is terminal (that is, it has no outgoing directed edges), we stop.

Now, say our starting node is eventually safe if and only if we must eventually walk to a terminal node.  More specifically, there exists a natural number K so that for any choice of where to walk, we must have stopped at a terminal node in less than K steps.

Which nodes are eventually safe?  Return them as an array in sorted order.

The directed graph has N nodes with labels 0, 1, ..., N-1, where N is the length of graph.  The graph is given in the following form: graph[i] is a list of labels j such that (i, j) is a directed edge of the graph.

Example:
Input: `graph = [[1,2],[2,3],[5],[0],[5],[],[]]`
Output: `[2,4,5,6]`

Ok coming off of the course schedule problem, this one was much more intuitive. it's basically the same, except this time whenever we find a cycle, we need to set every single node on that cycle's path to unsafe. This means adding a third state on top of VISITING and VISITED to our status options, one for UNSAFE. now if we ever hit a node that is either in state visiting _or_ state unsafe, we have to set every node in the current path to unsafe. We also need to keep going even if we find a cycle, until every node has been checked by the top loop. Leetcode timed out when I wasn't keeping separate track of the current path, which makes sense because without it you have to loop through the entire status dict every time you need to update a path. so instead, every time you start a new node, a new current path is begun. you add each node to that path as you are VISITING it, and then remove it from the path once it's been VISITED. then if a cycle is found, you loop through all nodes in the current path and set them all to UNSAFE and start anew with the next node. then you can build the list of nodes in your status dictionary that ended up VISITED rather than UNSAFE and you have your eventually safe nodes!

Again the runtime for this is the basic dfs O(V + E) with some additional loops that don't up that runtime. If there are many cycles, you will have to loop over nodes more than once, but not a multiplicative number of times. Space complexity is only larger for tracking the current path and the list of eventual safe states. But these should only get larger at a constant x E so we're good there too.

In [ ]:
class Solution:
    def eventualSafeNodes(self, graph: List[List[int]]) -> List[int]:
        n = len(graph)
        self.graph = graph
        self.VISITING, self.VISITED, self.UNSAFE = -1, 0, 1
        self.status = {}
        for i in range(n):
            currPath = []
            if self.hasCycle(i, currPath):
                for node in currPath:
                    self.status[node] = self.UNSAFE
        return [x for x in self.status if self.status[x] == self.VISITED]
    
    def hasCycle(self, i, currPath):
        if i in self.status:
            if self.status[i] == self.VISITED:
                return False
            return True
        self.status[i] = self.VISITING
        currPath.append(i)
        for node in self.graph[i]:
            if self.hasCycle(node, currPath):
                return True
        self.status[i] = self.VISITED
        currPath.remove(i)
        return False
        
                    

**Circular Array Loop**

You are given a circular array nums of positive and negative integers. If a number k at an index is positive, then move forward k steps. Conversely, if it's negative (-k), move backward k steps. Since the array is circular, you may assume that the last element's next element is the first element, and the first element's previous element is the last element.

Determine if there is a loop (or a cycle) in nums. A cycle must start and end at the same index and the cycle's length > 1. Furthermore, movements in a cycle must all follow a single direction. In other words, a cycle must not consist of both forward and backward movements.

Hmm. So the big question here is how do we know when to stop when we _don't_ find a cycle? The only 2 scenarios I can think of where that would happen are when the direction changes, or if an int ever points back to itself. so we need to keep track of the current path's direction and then check if the direction has changed at the start of the `hasCycle` function. Then if the current node IS in visiting state, we need to loop through the status dictionary and only return true if more than one node is in that state. The only other change we need to consider is that each int only points to one other, so instead of looping through that node's neighbors we just need to calculate the node its pointing to (and mod it by the length of the list). this is where we can also make sure that we return false if a node points to itself-- if this new index == the current index, don't recurse. 

Interestingly, the runtime of this is actually smaller than the classic dfs O(V + E) because each node only has one edge, so E == V, which means the runtime is V + V ==> O(V). Same with the space complexity. 

In [ ]:
class Solution:
    def circularArrayLoop(self, nums: List[int]) -> bool:
        self.n = len(nums)
        self.nums = nums
        self.status = {}
        self.VISITING, self.VISITED = -1, 0
        for i in range(self.n):
            if nums[i] < 0:
                direction = -1
            else:
                direction = 1
            if self.hasCycle(i, direction):
                return True
        return False
    
    def hasCycle(self, i, direction):
        if (self.nums[i] < 0 and direction > 0) or (self.nums[i] > 0 and direction < 0):
            return False
        if i in self.status:
            if self.status[i] == self.VISITING:
                count = 0
                for node in self.status:
                    if self.status[node] == self.VISITING:
                        count += 1
                    if count > 1:
                        return True
            return False
        self.status[i] = self.VISITING
        j = (i + self.nums[i]) % self.n
        if j != i and self.hasCycle(j, direction):
            return True
        self.status[i] = self.VISITED
        return False
                    

**Number of Islands**

Given a 2d grid map of '1's (land) and '0's (water), count the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

This seems to be super similar to the rotting oranges problem, just counting the starting situation. if you find a '1', bfs for all the '1's around it and then add one to the count when it terminates. return the count once you've checked every starting node. I started doing this with a `seen` object tracking which nodes we've seen, but this exceeded the time limit. I then realized we could just manipulate the grid itself to track what we needed to (I generally don't like doing this in order to preserve the inputs), and when I removed `seen` and simply updated a node's value to '0' after it was counted, it passed.

the solution is a classic BFS, which when used on a graph given as a matrix is O(m * ) with m and n being the number of columns and rows, respectively. The space complexity is O(min(m, n)) because the queue can only grow up to the min of those two before items get removed.

In [ ]:
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if not grid: return 0
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.dirs = [(1,0), (-1,0), (0,1),(0,-1)]
        count = 0
        for i in range(self.n):
            for j in range(self.m):
                if self.isIsland(i, j):
                    count += 1
        return count
    
    def isIsland(self, i, j):
        if self.grid[i][j] == '0': return False
        q = [(i,j)]
        self.grid[i][j] = '0'
        while q: 
            x,y = q.pop(0)
            for xD, yD in self.dirs:
                neiX = x + xD
                neiY = y + yD
                if 0 <= neiX < self.n and 0 <= neiY < self.m:
                    if self.grid[neiX][neiY] == '1':
                        self.grid[neiX][neiY] = '0'
                        q.append((neiX, neiY))
        return True

**Knight's Tour**

You are given a rows * cols chessboard and a knight that moves like in normal chess. 
Currently knight is at starting position denoted by start_row th row and start_col th col, and want to reach at ending position denoted by end_row th row and end_col th col. The goal is to calculate the minimum number of moves that the knight needs to take to get from starting position to ending position.
start_row, start_col, end_row and end_col are 0-indexed. 

Input Format:
There are six arguments. First is an integer denoting rows, second is an integer denoting cols, third is an integer denoting start_row, fourth is an integer denoting start_col, fifth is an integer denoting end_row and sixth is an integer denoting end_col.

Output Format:
Return an integer. If it is possible to reach from starting position to ending position then return minimum number of moves that the knight needs to take to get from starting position to ending position. If it is not possible to reach from starting position to ending position then return -1.

Ok it's looking for the shortest path so I want to go with BFS yet again. We don't even need to loop through each node as a start because we're given the starting node. so we just need to bfs from that starting node, keeping track of the number of moves at each step. compare each node we land at to the end node and return the # of moves there if we hit it. if the bfs ends and we haven't found the target, return -1.

This is another classic BFS except that we only ever start from a single node. So it's _at most_ O(m * n) but stops when it finds the shortest path. I believe space is O(min(m, n)) again just like above, because even though the neighbors of a knight's node are 8, they still span out in the same way and are limited by the number of rows and cols in the same way.

This exceeded the time limit on `interviewbit.com`, but did not give the input that broke the code, so I'm not sure why it did. The problem itself has been removed from leetcode.

In [ ]:
class Solution:
    def knight(self, rows, cols, start_r, start_c, end_r, end_c):
        self.rows, self.cols = rows, cols
        self.start_r, self.start_c = start_r, start_c
        self.end_r, self.end_c = end_r, end_c
        return self.bfs()
    def bfs(self):
        seen = set()
        dirs = [(1,2),(-1,2),(-2,1),(-2,-1),(-1,-2),(1,-2),(2,-1),(2,1)]
        q = [(self.start_r, self.start_c, 0)]
        while q:
            x, y, moves = q.pop(0)
            if (x,y) == (self.end_r, self.end_c):
                return moves
            seen.add((x,y))
            for xD, yD in dirs:
                neiX = x + xD
                neiY = y + yD
                if (neiX, neiY) not in seen and 0 <= neiX < self.rows and 0 <= neiY < self.cols:
                    q.append((neiX, neiY, moves + 1))
        return -1

**Clone Graph**

Given a reference of a node in a connected undirected graph, return a deep copy (clone) of the graph.

Each node in the graph contains a val (int) and a list (List[Node]) of its neighbors.
```
class Node {
    public int val;
    public List<Node> neighbors;
}
``` 

Test case format:
For simplicity sake, each node's value is the same as the node's index (1-indexed). For example, the first node with val = 1, the second node with val = 2, and so on. The graph is represented in the test case using an adjacency list.

Adjacency list is a collection of unordered lists used to represent a finite graph. Each list describes the set of neighbors of a node in the graph.

The given node will always be the first node with val = 1. You must return the copy of the given node as a reference to the cloned graph.

First thing's first, we need to traverse the graph, so either a dfs or bfs should work. for each new node on the stack, if it hasn't been seen yet we need to create a new node that references it, and then loop through its neighbors to create new nodes for each of them. But we need to track the new nodes somehow to ensure that we don't create multiple new nodes for a single value that has multiple neighbors. so let's keep the new nodes in a dictionary of node.val: node so that when we get to a node _in_ seen, we can easily return that node's clone instead. And in fact, we can check the dictionary _instead_ of a seen set. 

This is a classic dfs that is ensured to be connected, which means that we don't do an initial loop through all of the nodes to make sure they're all hit. If there were no edges, we would not still loop through multiple nodes: to remain connected, a graph with no edges would be only a single node. so I believe the run time is actually O(E), E being the number of edges. Technically it's E + 1, but it reduces. Space-wise, the visited dictionary is O(V + E).

In [ ]:
"""
# Definition for a Node.
class Node:
    def __init__(self, val, neighbors):
        self.val = val
        self.neighbors = neighbors
"""
class Solution:
    def cloneGraph(self, node: 'Node') -> 'Node':
        if not node: return 
        self.visited = {}
        return self.dfs(node)
    
    def dfs(self, node):
        if node.val not in self.visited:
            newNode = Node(node.val, [])
            self.visited[node.val] = newNode
            for nei in node.neighbors:
                newNode.neighbors.append(self.dfs(nei))
            return newNode
        return self.visited[node.val]
        

**Course Schedule II (return a valid order)**

There are a total of n courses you have to take, labeled from 0 to n-1.


Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: [0,1]


Given the total number of courses and a list of prerequisite pairs, return the ordering of courses you should take to finish all courses.


There may be multiple correct orders, you just need to return one of them. If it is impossible to finish all courses, return an empty array.

So this is the same algorithm, just tracking the order as you go. The original way i did it with each class pointing to the classes that rely on _it_ would require reversing the list at the end, so we can swap the order and have each class point to all of its prerequisites, and return the order as it was built.

In [ ]:
class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        if numCourses < 1: return []
        self.status = {}
        self.VISITING = -1
        self.VISITED = 0
        self.order = []
        self.preqs = {i: [] for i in range(numCourses)}
        for preq in prerequisites:
            self.preqs[preq[0]].append(preq[1])
        for i in range(numCourses):
            if self.hasCycle(i):
                return []
        return self.order
    
    def hasCycle(self, node):
        if node in self.status:
            if self.status[node] == self.VISITING:
                return True
            return
        self.status[node] = self.VISITING
        for req in self.preqs[node]:
            if self.hasCycle(req):
                return True
        self.status[node] = self.VISITED
        self.order.append(node)
        return

**BST to Greater Sum Tree**

Given the root of a binary search tree with distinct values, modify it so that every node has a new value equal to the sum of the values of the original tree that are greater than or equal to node.val.

So the thing to note is that this requires a backward inorder traversal. If we keep track of a `total` as we traverse from right to left, we can add the current node's value to that total, and _then_ update the node's value to the new total. Then we can recurse left. Before we sum though, we have to make sure we start at the right place, ie we don't want the sum to change from 0 _until_ we reach the rightmost node. so we recurse right, update total, update node.val, recurse left.

The runtime of this is the same as a classic inorder traversal, so O(n) for the number of nodes in the tree. Space used is no more than the tree itself plus the recursion stack, so O(n) for the stack. 

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution:
    def bstToGst(self, root: TreeNode) -> TreeNode:
        self.total = 0
        self.backwardTraversal(root)
        return root
    
    def backwardTraversal(self, node):
        if node:
            self.backwardTraversal(node.right)
            self.total += node.val
            node.val = self.total
            self.backwardTraversal(node.left)

**Alien Dictionary**

There is a new alien language which uses the latin alphabet. However, the order among letters are unknown to you. You receive a list of non-empty words from the dictionary, where words are sorted lexicographically by the rules of this new language. Derive the order of letters in this language.

**Note**

1. You may assume all letters are in lowercase.
2. You may assume that if a is a prefix of b, then a must appear before b in the given dictionary.
3. If the order is invalid, return an empty string.
4. There may be multiple valid order of letters, return any one of them is fine.

Um this is hard. haha! So we definitely get some clues about what type of question this is, in that letters are going to have to point to letters that come after them in the alien dict, making this a graph problem. We're also told that we're looking for the order of the nodes (and if there is no valid order, i.e. if there is a cycle to return basically null), which requires a topological sort. So the trick is figuring out how to build the adjacency lists. we need each letter (node) to point to the letters (nodes) that we know come after it in the dictionary. since the words are sorted by their first letters, we can do part of that work by comparing the first letter in each word going through the list in order. however if there are multiple words that begin with the same letter/s, we will need to loop through them to compare the first non-equal letter in those words. Once we find the first non-equal letter, we must break out of that comparison, because the letters _within_ a given word are not sorted. There are a few special quirks to this we need to keep in mind. We'll need to loop through all the words but compare one to the word next to it, so we'll want to loop (# of words - 1) times. within each comparison, we'll want to loop until we find a character **not** in common, but we'll need to ensure that we don't try to check past the length of the smaller of the two words. So this inner loop will only want to go for (min(len(word1), len(word2)) times. Once we have our adjacency list, however, it is a classic topological sort with a cycle check. 

In [ ]:
class Solution:
    def alienOrder(self, words: List[str]) -> str:
        self.adj_list = self.buildGraph(words)
        self.order = []
        self.seen = {}
        self.VISITING = -1
        self.VISITED = 0
        for word in words:
            for ch in word:
                if self.buildValidOrder(ch) == -1:
                    return ""
        self.order.reverse()
        return "".join(self.order)
         
    def buildGraph(self, words):
        adj_list = {ch: set() for word in words for ch in word}
        num_words = len(words)
        for i in range(num_words - 1):
            min_chars = min(len(words[i]), len(words[i + 1]))
            for j in range(min_chars):
                if words[i][j] != words[i + 1][j]:
                    adj_list[words[i][j]].add(words[i + 1][j])
                    break
        return adj_list
        
    def buildValidOrder(self, node):
        if node in self.seen:
            if self.seen[node] == self.VISITING:
                return -1
            return
        
        self.seen[node] = self.VISITING
        for nei in self.adj_list[node]:
            if self.buildValidOrder(nei) == -1:
                return -1
            
        self.seen[node] = self.VISITED
        self.order.append(node)